In [ ]:
# ------------------------------------------------ #
#                  IMPORT FUNCTIONS                #
# ------------------------------------------------ #
import os
import sys
import datetime

# Get the project root directory (going up one level from Notebooks)
project_root = os.path.dirname(os.getcwd())
# Add the Scripts directory to Python path so we can import modules from individual scripts files
scripts_dir = os.path.join(project_root, 'Scripts')
if scripts_dir not in sys.path:
    sys.path.append(scripts_dir)

# Add functions here from Scripts folder
from ao_indices import return_AO_fetch
from enso_indices import return_ENSO_fetch
from breakup_data import calculate_breakup_data
from breakup_data import calculate_estimated_breakup
from bounding_box import calculate_bounding_box
from raster_temp import mean_30d_raster_temps
#from raster_temp_era5 import mean_30d_raster_temps
from correlation_function import (
    calculate_ao_correlation_per_month,
    calculate_enso_correlation_per_month,
    plot_heatmap,
    plot_yearly_correlation,
    save_results_to_csv
)


In [ ]:
# ------------------------------------------------ #
#                CLASS DEFINITION                  #
# ------------------------------------------------ #

class StudySite:
    def __init__(self,name):
        self.name = name
        self.breakup_anomaly_data = None # Dictionary of integer years (e.g. 2000) with following data subkeys: zscore_index, anomaly_days, breakup_date, breakup_doy
        self.estimated_breakup_doy = None # The MEAN breakup date for the study site across all years, in DOY integer format.
        self.enso_indices = None # Dictionary of integer years (e.g. 2000) with enso_value for each of 12 months before breakup, named as strings
        self.ao_indices = None # Dictionary of integer years (e.g. 2000) with ao_value for each of 12 months before breakup
        self.bounding_box = None  # Dictionary of max_lat:value, min_lat:value, max_lon:value, min_lon:value
        self.mean_30d_raster_temps = None # Dictionary of integer years (e.g. 2000) with rasters of the mean breakup temperature across the 30 days prior to mean breakup date (we can't do actual breakup date because months have different temps)
        self.enso_correlations = None # Single raster of ENSO correlations across all years
        self.ao_correlations = None # Single raster of AO correlations across all years


    def calculate_breakup_data(self, ice_data_filepath, years_of_data):
        """ Load ice breakup data for this site from the provided icedata spreadsheet, filtering by years_of_data and the site name """

        # Dictionary of integer years (e.g. 2000) with following data subkeys: zscore_index, anomaly_days, breakup_doy
        self.breakup_anomaly_data = calculate_breakup_data(years_of_data, ice_data_filepath, self.name)

        # Single value of estimated breakup day of year for this site (MEAN across all years)
        self.estimated_breakup_doy = calculate_estimated_breakup(self.breakup_anomaly_data)


    def fetch_enso(self,enso_data_filepath):
        """ Pull in ENSO data as dictionary of integer years (e.g. 2000) on a MONTHLY scale for 12 months prior to estimated breakup 
            date from all years of data (excluding month with breakup date, as we are looking for time lag of 1+ months) """
        
        # Dictionary of integer years (e.g. 2000) with dictionary of preceding 12 month_name: month_values
        self.enso_indices = return_ENSO_fetch(enso_data_filepath, self.breakup_anomaly_data, self.estimated_breakup_doy)


    def fetch_ao(self,ao_data_filepath):
        """ Pull in AO data as dictionary of integer years (e.g. 2000) on a MONTHLY scale for 12 months prior to estimated breakup 
            date from all years of data (excluding month with breakup date, as we are looking for time lag of 1+ months) """
        
        # Dictionary of integer years (e.g. 2000) with dictionary of preceding 12 month_name: month_values
        self.ao_indices = return_AO_fetch(ao_data_filepath, self.breakup_anomaly_data, self.estimated_breakup_doy)


    def calculate_bounding_box(self, ice_data_filepath):
        """ Calculate 100km bounding box based on study site coordinates """

        # Bounding box for this site based on lat,lon for each site in the ice data CSV
        # Dictionary of max_lat:value, min_lat:value, max_lon:value, min_lon:value
        self.bounding_box = calculate_bounding_box(ice_data_filepath, self.name)


    def calculate_mean_raster_temps(self):
        """ Pull the raster data within the bounding box for each of the 30d prior to the given breakup date variable, then average all the rasters together to return the mean 30d temp for each year."""

        # Dictionary of integer years (e.g. 2000) paired with rasters of the mean breakup temperature across the 30 days prior to given breakup date variable for each year
        # (Note we can't do actual breakup date for each year for each site due to statistical reasons, so for each site we will find the mean breakup date and use that to find our 30d period)
        self.mean_30d_raster_temps = mean_30d_raster_temps(self.bounding_box, self.breakup_anomaly_data, self.estimated_breakup_doy)
       
    # For each month in our indices in the 12 months prior to breakup (excluding breakup month,) calculate the P value across all years of data
    # Single raster for each of these consisting of correlation across all years
   def calculate_ao_correlation(self, ao_indices, years):
    """
    Calculate AO correlations per month for this study site.
    """
    self.ao_correlations = calculate_ao_correlation_per_month(
        ao_indices=ao_indices,
        mean_30d_raster_temps=self.mean_30d_raster_temps,
        community_names=[self.name],
        years=years
    )


   def calculate_enso_correlation(self, enso_indices, years):
    """
    Calculate ENSO correlations per month for this study site.
    """
    self.enso_correlations = calculate_enso_correlation_per_month(
        enso_indices=enso_indices,
        mean_30d_raster_temps=self.mean_30d_raster_temps,
        community_names=[self.name],
        years=years
    )


In [ ]:
# ------------------------------------------------ #
#                  PARAMETERS                      #
# ------------------------------------------------ #

# Add data here from Data folder
enso_data_filepath = os.path.join(project_root, "Data", "ENSO_index.txt")
ao_data_filepath = os.path.join(project_root, "Data", "AO_index.txt")
ice_data_filepath = os.path.join(project_root, "Data", "ice_data.csv")
results_dir = os.path.join(project_root, "Results")
os.makedirs(results_dir, exist_ok=True)

# Choose study sites here
study_site_names = ["Stebbins"]

# Generate a list of years from 2000 to 2022 - select years
years_of_data = list(range(2000, 2023))

In [ ]:
# ------------------------------------------------ #
#                  MAIN CODE                      #
# ------------------------------------------------ #

# Create list of StudySite objects
sites = [StudySite(name) for name in study_site_names]

# Process data for each site
for site in sites:
    # Calculate basic site data
    site.calculate_breakup_data(ice_data_filepath, years_of_data)
    
    # Fetch AO and ENSO indices
    site.fetch_ao(ao_data_filepath)
    site.fetch_enso(enso_data_filepath)
      
    # Calculate bounding box
    site.calculate_bounding_box(ice_data_filepath)
    
    # Calculate mean 30-day raster temperatures
    site.calculate_mean_raster_temps()
    
    # Calculate AO correlations (per month and spatial)
    site.calculate_ao_correlation(
        ao_indices=site.ao_indices,
        years=years_of_data
    )
    
    # Calculate ENSO correlations (per month and spatial)
    site.calculate_enso_correlation(
        enso_indices=site.enso_indices,
        years=years_of_data
    )
    
    print(f"Finished processing site: {site.name}")
    print(f"Breakup anomaly data: {site.breakup_anomaly_data}")
    print(f"AO Correlation Results: {site.ao_correlations}")
    print(f"ENSO Correlation Results: {site.enso_correlations}")


In [ ]:
# ------------------------------------------------ #
#       CORRELATION AND PLOTTING (Optional)        #
# ------------------------------------------------ #

# Save and plot AO and ENSO correlations for each site
for site in sites:
    if site.ao_correlations is not None:
        # Save AO correlations to CSV
        ao_csv_path = os.path.join(results_dir, f"ao_correlation_{site.name}.csv")
        save_results_to_csv(site.ao_correlations, ao_csv_path)
        print(f"AO correlation results saved for {site.name} at {ao_csv_path}")
        
        # Plot AO yearly line graphs
        ao_linegraph_dir = os.path.join(results_dir, "AO_LineGraphs")
        plot_yearly_correlation(
            results=site.ao_correlations, 
            community_name=site.name, 
            variable="AO", 
            output_dir=ao_linegraph_dir
        )
        print(f"AO yearly line graph plotted for {site.name}")

        # Plot AO spatial heatmaps
        ao_heatmap_dir = os.path.join(results_dir, "AO_Heatmaps")
        plot_heatmap(
            correlation_raster=site.ao_correlations, 
            community_name=site.name, 
            variable="AO", 
            output_dir=ao_heatmap_dir
        )
        print(f"AO heatmap plotted for {site.name}")
    
    if site.enso_correlations is not None:
        # Save ENSO correlations to CSV
        enso_csv_path = os.path.join(results_dir, f"enso_correlation_{site.name}.csv")
        save_results_to_csv(site.enso_correlations, enso_csv_path)
        print(f"ENSO correlation results saved for {site.name} at {enso_csv_path}")
        
        # Plot ENSO yearly line graphs
        enso_linegraph_dir = os.path.join(results_dir, "ENSO_LineGraphs")
        plot_yearly_correlation(
            results=site.enso_correlations, 
            community_name=site.name, 
            variable="ENSO", 
            output_dir=enso_linegraph_dir
        )
        print(f"ENSO yearly line graph plotted for {site.name}")

        # Plot ENSO spatial heatmaps
        enso_heatmap_dir = os.path.join(results_dir, "ENSO_Heatmaps")
        plot_heatmap(
            correlation_raster=site.enso_correlations, 
            community_name=site.name, 
            variable="ENSO", 
            output_dir=enso_heatmap_dir
        )
        print(f"ENSO heatmap plotted for {site.name}")
